In [1]:
!pip install mealpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.3/386.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 39.8 MB/s eta 0:00:00


In [14]:
from mealpy import Problem, FloatVar, BBO, IntegerVar, GA, StringVar
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from mealpy import GA, PSO, Problem, FloatVar, IntegerVar
import numpy as np

SEED = 42


class Squared(Problem):
    def __init__(self, bounds=None, minmax="min", name="Squared", data=None, **kwargs):
        self.name = name
        self.data = data
        super().__init__(bounds, minmax, **kwargs)

    def obj_func(self, solution):
        x = self.decode_solution(solution)["weight"]
        b = self.decode_solution(solution)["base"]
        return [x**2, b**2]

## Now, we define an algorithm, and pass an instance of our *Squared* class as the problem argument.
bounds = [IntegerVar(lb=10, ub=100, name="weight"), IntegerVar(lb=10, ub=20, name="base")]
problem = Squared(bounds=bounds, minmax="min", name="Squared", data="Amazing", obj_weights=(0.5, 0.5))
model = BBO.OriginalBBO(epoch=100, pop_size=20, seed=SEED)
model_2 = GA.BaseGA(epoch=100, pop_size=100)
g_best = model.solve(problem)

INFO:mealpy.bio_based.BBO.OriginalBBO:Solving 2-objective optimization problem with weights: [0.5 0.5].
INFO:mealpy.bio_based.BBO.OriginalBBO:>>>Problem: P, Epoch: 1, Current best: 145.0, Global best: 145.0, Runtime: 0.00347 seconds
INFO:mealpy.bio_based.BBO.OriginalBBO:>>>Problem: P, Epoch: 2, Current best: 121.0, Global best: 121.0, Runtime: 0.00548 seconds
INFO:mealpy.bio_based.BBO.OriginalBBO:>>>Problem: P, Epoch: 3, Current best: 121.0, Global best: 121.0, Runtime: 0.00580 seconds
INFO:mealpy.bio_based.BBO.OriginalBBO:>>>Problem: P, Epoch: 4, Current best: 121.0, Global best: 121.0, Runtime: 0.00555 seconds
INFO:mealpy.bio_based.BBO.OriginalBBO:>>>Problem: P, Epoch: 5, Current best: 121.0, Global best: 121.0, Runtime: 0.00567 seconds
INFO:mealpy.bio_based.BBO.OriginalBBO:>>>Problem: P, Epoch: 6, Current best: 121.0, Global best: 121.0, Runtime: 0.00613 seconds
INFO:mealpy.bio_based.BBO.OriginalBBO:>>>Problem: P, Epoch: 7, Current best: 121.0, Global best: 121.0, Runtime: 0.00679 s

In [ ]:
g_best

id: 4727, target: Objectives: [100 100], Fitness: 100.0, solution: [10. 10.]

In [ ]:
getattr(tf.k, "Adam")

AttributeError: module 'tensorflow' has no attribute 'k'

In [ ]:
model = Sequential([
    Dense(1, activation='linear'),
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
               optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
               metrics=['accuracy'])

In [ ]:
class LogisticRegression(Problem):
    def __init__(self, bounds=None, minmax="min", name="LogisticRegression", data=None, **kwargs):
        self.name = name
        self.data = data
        super().__init__(bounds, minmax, **kwargs)

    def obj_func(self, solution):
        x = self.decode_solution(solution)["learning_rate"]
        model = Sequential([
            Dense(1, activation='linear'),
        ])
        model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
               optimizer=tf.keras.optimizers.Adam(learning_rate=x),
               metrics=['accuracy'])
        model.fit(X_train, y_train, epochs=50)
        acc = model.evaluate(X_test, y_test)
        return acc[1]

In [8]:
class LogisticRegression():
    def __init__(self, input: int, optimizer: str, learning_rate: float):
        self.input = input
        self.optimizer_name = optimizer
        self.learning_rate = learning_rate
        self.sequential = Sequential([
            Input(shape=(input,)),
            Dense(1, activation='linear'),
        ])

    def fit(self, X, y, epochs=50):
        self.sequential.compile(
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=getattr(tf.keras.optimizers, self.optimizer_name)(learning_rate=self.learning_rate))
        self.sequential.fit(X, y, epochs=epochs)
        loss = self.sequential.evaluate(X, y)
        return loss

    def predict(self, X, y):
        test_loss = self.sequential.evaluate(X, y)
        return test_loss



In [13]:
class HyperparameterOptimProblem(Problem):
    def __init__(self, bounds, minmax="min", data=None, **kwargs):
        self.data = data

        super().__init__(bounds, minmax, **kwargs)

    def fit_model(self, optimizer_name, learning_rate):
        # Create Logistic Regression model with current hyperparameters
        input_dim = self.data["X_train"].shape[1]
        model = LogisticRegression(input=self.input_dim, optimizer=optimizer_name, learning_rate=learning_rate)

        # Train the model and return loss as the evaluation metric
        loss_train = model.fit(self.data["X_train"], self.data["y_train"], epochs=self.data["epoch"])
        loss_test = model.predict(self.data["X_test"], self.data["y_test"])
        return [loss_train, loss_test]

    def obj_func(self, solution) -> list[float, float]:
        # Decode the solution to get optimizer index and learning rate
        optimizer_name, learning_rate = solution["optimizer"], solution["learning_rate"]
        train_test_loss = self.fit_model(optimizer_name, learning_rate)  # Evaluate model
        return train_test_loss

In [9]:
data = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.33, random_state=42)

In [ ]:
data = {
    "X_train": X_train,
    "y_train": y_train,
    "X_test": X_test,
    "y_test": y_test,
    "epoch": 50
}
optimizers = [ "SGD", "Adam", "RMSprop", "Adagrad", "Adadelta", "Adamax"]
bounds = [
    StringVar(lb=0, ub=1, name="optimizer"),
    FloatVar(lb=0.001, ub=0.1, name="learning_rate"),
]

In [10]:
model1 = LogisticRegression(input=30, optimizer="Adam", learning_rate=0.01)
model1.fit(X_train, y_train)

Epoch 1/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 123.9464  
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 29.8204 
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 20.9765 
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 13.0495 
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.7051  
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6.2935 
Epoch 7/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.1457 
Epoch 8/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.7785 
Epoch 9/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.8632 
Epoch 10/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.4182 
Epoch 11/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.5659 
Epoch 12/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.3421 
Epoch 13/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.9556 
Epoch 14/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.8834 
Epoch 15/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.457

0.18864493072032928

In [11]:
model1.predict(X_test, y_test)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0852 


0.07767941057682037

In [ ]:
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5979 - loss: 233.6984  
Epoch 2/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5888 - loss: 102.9982 
Epoch 3/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3018 - loss: 29.8380  
Epoch 4/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3391 - loss: 26.1600  
Epoch 5/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3591 - loss: 21.0172 
Epoch 6/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2858 - loss: 16.2895  
Epoch 7/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2825 - loss: 11.2856  
Epoch 8/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3429 - loss: 6.9593  
Epoch 9/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4392 - loss: 3.9857  
Epoch 10/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5527 - loss: 2.5533  
Epoch 11/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6381 - loss: 1.5043  
Epoch 12/100
12/12 ━━━━━━━━━━━━━━━